In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
import re
import string
import unicodedata
from sklearn.feature_extraction.text import  CountVectorizer
import scipy.stats as stats
from bs4 import BeautifulSoup
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem import WordNetLemmatizer


In [3]:
df_train = pd.read_csv("sentiment_train (1).csv")

print(df_train.info())
print(df_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB
None
                                            Sentence  Polarity
0                           Wow... Loved this place.         1
1                                 Crust is not good.         0
2          Not tasty and the texture was just nasty.         0
3  Stopped by during the late May bank holiday of...         1
4  The selection on the menu was great and so wer...         1


In [4]:
df_test = pd.read_csv("sentiment_test.csv")

print(df_test.info())
print(df_test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  600 non-null    object
 1   Polarity  600 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ KB
None
                                            Sentence  Polarity
0  A good commentary of today's love and undoubte...         1
1  For people who are first timers in film making...         1
2  It was very popular when I was in the cinema, ...         1
3  It's a feel-good film and that's how I felt wh...         1
4  It has northern humour and positive about the ...         1


In [5]:
stop_words = set(stopwords.words('english')) -{'not', 'no', "wouldn't", "couldn't", "aren't", "below"}
print(stop_words)

{'weren', 'shouldn', 'yours', 'her', 'which', 'how', 'very', 'been', 'shan', 's', 'between', 'where', 'o', "weren't", 'up', 'are', 'being', 'about', 'a', 'haven', 'to', "isn't", 'he', 'did', 'its', 'my', 'we', 'm', 'hasn', 'before', 'you', 'yourselves', 'hadn', 'their', 'wasn', "mustn't", 'theirs', 'this', 'didn', 'above', 'as', 'do', 'aren', "mightn't", "won't", 'itself', "it's", 'once', 'the', 'all', 'over', 'me', 'with', 'll', 'having', 'further', 'than', 'while', 'in', 'then', 'more', 'other', 'only', 't', 'both', "that'll", 'y', 'ourselves', "you're", 'few', 'don', 'who', 'most', 'mightn', "needn't", 'our', 'if', "haven't", 'an', 'whom', 'needn', 're', 'himself', 'or', 'that', 'now', 'ain', 'them', 'from', 'some', 'does', 'themselves', 'why', 'during', 'down', 'into', "don't", 'out', 'isn', "you've", 'after', 'at', "doesn't", 'yourself', 'same', 'again', "she's", 'mustn', "shouldn't", "you'd", 'against', 'these', 'wouldn', 'and', 'off', 'is', 'ours', 'what', 'can', 'ma', 'was', 'b

In [6]:
lemmer = WordNetLemmatizer()

In [7]:
def preprocess(x):
    # Lower case
    x = x.lower()
    
    # Remove HTML tags
    x = BeautifulSoup(x, "lxml").get_text()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s]', '', x)

    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
     # Remove URLs
    x = re.sub(r'http\S+', '', x)
 
    # Remove single characters from the start
    x = re.sub(r'\^[a-zA-Z]\s+', ' ', x) 
    
    
     # Substituting multiple spaces with single space
    x = re.sub(r'\s+', ' ',x, flags=re.I)

    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 
 

df_train['Sentence_pre'] = df_train['Sentence'].apply(preprocess)

df_test['Sentence_pre'] = df_test['Sentence'].apply(preprocess)

In [30]:
print(df_train.head())
print(df_test.head())

                                            Sentence  Polarity  \
0                           Wow... Loved this place.         1   
1                                 Crust is not good.         0   
2          Not tasty and the texture was just nasty.         0   
3  Stopped by during the late May bank holiday of...         1   
4  The selection on the menu was great and so wer...         1   

                                        Sentence_pre  
0                                    wow loved place  
1                                     crust not good  
2                            not tasty texture nasty  
3  stopped late may bank holiday rick steve recom...  
4                         selection menu great price  
                                            Sentence  Polarity  \
0  A good commentary of today's love and undoubte...         1   
1  For people who are first timers in film making...         1   
2  It was very popular when I was in the cinema, ...         1   
3  It's a

In [56]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
    
    
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words= stop_words)

ngram_vectorizer.fit(df_train["Sentence_pre"])

X = ngram_vectorizer.transform(df_train["Sentence_pre"])

X_test = ngram_vectorizer.transform(df_test["Sentence_pre"])
y = df_train['Polarity']
y_test=df_test['Polarity']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75)





CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', 'as', 'at', 'be', 'because', 'been',
                            'before', 'being', 'between', 'both', 'but', 'by',
                            'can', 'couldn', 'd', 'did', 'didn', ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [57]:
for c in [0.01]:
    svm1 = LinearSVC(C=c)
svm1.fit(X_train, y_train)

print ("Accuracy for C=%s: %s"
       % (c, accuracy_score(y_val, svm1.predict(X_val))))

for c in [0.05]:
    svm2 = LinearSVC(C=c)
svm2.fit(X_train, y_train)

print ("Accuracy for C=%s: %s"
       % (c, accuracy_score(y_val, svm2.predict(X_val))))

for c in [0.25]:
    svm3 = LinearSVC(C=c)
svm3.fit(X_train, y_train)

print ("Accuracy for C=%s: %s"
       % (c, accuracy_score(y_val, svm3.predict(X_val))))

for c in [0.5]:
    svm4 = LinearSVC(C=c)
svm4.fit(X_train, y_train)

print ("Accuracy for C=%s: %s"
       % (c, accuracy_score(y_val, svm4.predict(X_val))))

for c in [1]:
    svm5 = LinearSVC(C=c)
svm5.fit(X_train, y_train)

print ("Accuracy for C=%s: %s"
       % (c, accuracy_score(y_val, svm5.predict(X_val))))



# Accuracy for C=0.01: 0.82
# Accuracy for C=0.05: 0.84167
# Accuracy for C=0.25: 0.8467
# Accuracy for C=0.5: 0.8467
# Accuracy for C=1: 0.845
    
    

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Accuracy for C=0.01: 0.805


LinearSVC(C=0.05, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Accuracy for C=0.05: 0.8266666666666667


LinearSVC(C=0.25, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Accuracy for C=0.25: 0.8233333333333334


LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Accuracy for C=0.5: 0.8216666666666667


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

Accuracy for C=1: 0.8183333333333334


In [58]:
    
final_svm_ngram = LinearSVC(C=0.05)
final_svm_ngram.fit(X, y)


LinearSVC(C=0.05, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [59]:

test_predictions=final_svm_ngram.predict(X_test)



print(confusion_matrix(y_test,test_predictions))
print(classification_report(y_test,test_predictions))



[[257  30]
 [110 203]]
              precision    recall  f1-score   support

           0       0.70      0.90      0.79       287
           1       0.87      0.65      0.74       313

    accuracy                           0.77       600
   macro avg       0.79      0.77      0.76       600
weighted avg       0.79      0.77      0.76       600

